In [1]:
# %matplotlib inline
# -*- coding: utf-8 -*-
from __future__ import division #~ Domysle dzielenie int jako liczb float
from igraph import *     #~ Do symulacji grafu
import random            #~ Do wybierania losowych elementow z listy
# from  matplotlib import rc
import itertools        #~ do list i iteratorow
import numpy as np        #~ Do np.arange
import time                #~ Do diagnostyki czasu wykonania
import os                #~ Do sprawdzania istnienia folderów
import gzip                #~ Do pakowania plikow wynikowych
import cPickle as pickle   #~ Do pakowania plikow wynikowych
# from FilesManagment import CheckFolder, CompressFile, CompressData, FindFileNumber, SaveToFileNumberOfFiles

In [2]:
import numpy as np        #~ Do np.arange

In [3]:
import sys
sys.path.append('/dmj/fizmed/pkowalczyk/now/licencjat')
import SimulateClique
reload(SimulateClique)

<module 'SimulateClique' from '/dmj/fizmed/pkowalczyk/now/licencjat/SimulateClique.pyc'>

In [3]:
# rc('font', family='Arial') #Plotowanie polskich liter

In [ ]:
# podstawowy skrypt do badania wlasnosci
#~ Definicje stalych symulacji
stg = {
    'CONST_CLIQUE'    : 3,      #~ Wielkosc kliki
    'CONST_VERTICES'  : 10000,    #~ Ilosc wezlow
    'CONST_SIM_COUNT' : 1,      #~ Ilosc powtorzen symulacji
    'CONST_PRINT'     : False,  #~ Czy drukowac magnetyzacje co CONST_VERTICES krokow?
    'CONST_TIME'      : True,   #~ Czy przeprowadzac i drukowac wyniki diagnostyki?
    'CONST_FOLDER'    : "",     #~ Nic nie robi
    'CONST_OVERRIDEN' : False,  #~ Czy ma nadpisywac pliki podczas zapisywania wynikow
    'CONST_COMPRESS'  : True,   #~ Czy ma kompresowac dane przez zapisem    
    'CONST_SIM_LONG'  : 14,     # ile wielkosci N ma liczyc
    'CONST_PATH_BASIC_FOLDER' : 'Wyniki_fazowe',
    'CONST_START_MAGNETIZATION' : 0.5
}

k = 36
stg['CONST_EDGES'] = int(round(k * stg['CONST_VERTICES'] // 2, 0)) #~ Ilosc polaczen
stg['stopienG']    = round(stg['CONST_EDGES']/stg['CONST_VERTICES']*2, 1)

START, STOP, STEP = 0.50, 0.50, 0.01
for p in list(np.arange(START,STOP + STEP,STEP)):
    stg['CONST_START_MAGNETIZATION'] = p
    SimulateClique.jedna_symulacja(stg)

print 'Koniec Programu'


Rozpoczynam symulacje nr: 0 dla: 
ilosci wierzcholkow: 10000
ilosci polaczen: 160000
sredniego stopnia: 32.0
ilosci klik: 5494
wielkosc klik: 3
start_mag: 0.5
Magnetyzacja koncowa 140100  :  0.5016


In [4]:
import ipyparallel
clients = ipyparallel.Client()
# dview = clients.direct_view()

In [5]:
dview = clients[:]

In [6]:
results = dview.map_sync(lambda x: int(1), range(8))
print results

[1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# with dview.sync_imports():
#     import SimulateClique #[1]
# from ipyparallel.util import interactive

%px %matplotlib inline
# import matplotlib
# dview.apply_sync(matplotlib.use, 'agg')

def ippsim(p):
    def simulation(x):
        stg = {
            'CONST_CLIQUE'    : 3,      #~ Wielkosc kliki
            'CONST_VERTICES'  : 10000,    #~ Ilosc wezlow
            'CONST_SIM_COUNT' : 2,      #~ Ilosc powtorzen symulacji
            'CONST_PRINT'     : False,  #~ Czy drukowac magnetyzacje co CONST_VERTICES krokow?
            'CONST_TIME'      : True,   #~ Czy przeprowadzac i drukowac wyniki diagnostyki?
            'CONST_FOLDER'    : "",     #~ Nic nie robi
            'CONST_OVERRIDEN' : False,  #~ Czy ma nadpisywac pliki podczas zapisywania wynikow
            'CONST_COMPRESS'  : True,   #~ Czy ma kompresowac dane przez zapisem    
            'CONST_SIM_LONG'  : 11,     # ile wielkosci N ma liczyc
            'CONST_PATH_BASIC_FOLDER' : 'now/Wyniki_fazowe',
            'CONST_START_MAGNETIZATION' : 0.5
        }

        k = 77
        stg['CONST_EDGES']  = int(round(k * stg['CONST_VERTICES'] // 2, 0)) #~ Ilosc polaczen
        stg['CONST_MEAN_k'] = round(stg['CONST_EDGES']/stg['CONST_VERTICES']*2, 1)    
        stg['CONST_START_MAGNETIZATION'] = p

        import sys
        sys.path.append('/dmj/fizmed/pkowalczyk/now/licencjat')
        import SimulateClique
        return SimulateClique.jedna_symulacja(stg)
    return simulation

START, STOP, STEP = 0.5050, 0.52, 0.0025
for p in np.arange(START,STOP + STEP,STEP):
    print p
    results = dview.map_sync(ippsim(p), range(8))
    print results

print 'Koniec Programu'

0.505
[True, True, True, True, True, True, True, True]
0.5075


In [11]:
ipyparallel.util

<module 'ipyparallel.util' from '/dmj/fizmed/pkowalczyk/.local/lib/python2.7/site-packages/ipyparallel/util.pyc'>

In [ ]:
np.array(np.random.random_sample(stg['CONST_VERTICES']) > (1-stg['CONST_START_MAGNETIZATION']), np.int).mean()

In [8]:
print '{:0<5}'.format(1.45)

1.450


In [23]:
from __future__ import division #~ Domysle dzielenie int jako liczb float
%matplotlib inline

import sys
import os
import json
sys.path.append('/dmj/fizmed/pkowalczyk/now/complex_networks_sim')
from FilesManagment import CheckFolder
import matplotlib.pyplot as plt

In [34]:
def plotuj(stg, data):
    fig = plt.figure()
    plt.plot(data[0],data[1], '--o')
    plt.ylim(0,1)
    plt.xlim(0,1)
    plt.ylabel('Prawdopodobienstwo uzyskanie koncowego stanu sieci *w gore*')
    plt.xlabel('Poczatkowa magnetyzacja, N=6000')
    fig.suptitle('Zaleznosc koncowej magnetyzacji od sredniego stopnia wiercholkow sieci')
    fig.savefig(os.path.join(stg['CONST_STANDARD_PATH_ANALYZE'],'faza_dla_k{}N{}.png'.format(stg['CONST_CLIQUE'],stg['CONST_VERTICES'])), dpi = 200)
    fig.clf()

def check_folder_k(spin, path_file, basic_dir, stg):
    path_k = os.path.join(basic_dir, path_file, 'k{}'.format(spin))
    result = 0
    if os.path.exists(path_k):
        for path_opis in filter(lambda name: name.endswith('.json'), os.listdir(path_k)):
            with open(os.path.join(path_k, path_opis), 'r') as f:
                dic = json.load(f)
            if dic['CONST_VERTICES'] == stg['CONST_VERTICES'] and dic['WYN_meanG'] == stg['WYN_meanG']:
                result += 1
    return result

def analyze(stg):
    stg['CONST_STANDARD_PATH_ANALYZE'] = os.path.join(stg['CONST_PATH_BASIC_FOLDER'], 'analyze')
    CheckFolder(stg['CONST_STANDARD_PATH_ANALYZE'])
    stg['CONST_SHORT_RAW_PATH'] = os.path.join(stg['CONST_PATH_BASIC_FOLDER'], 'RawDataMag', 'klik' + str(stg['CONST_CLIQUE']), 'mag_start')

    x, y = [], []
    basic_dir = stg['CONST_SHORT_RAW_PATH']
    for path_file in sorted(os.listdir(basic_dir)):
        down = check_folder_k(0, path_file, basic_dir, stg)
        up   = check_folder_k(1, path_file, basic_dir, stg)
        if up + down != 0:
            x.append(float(path_file))
            y.append(up/(up+down))  

    wynik = (x,y)
    print wynik
    if stg['CONST_DUMP']:
        with open(os.path.join(stg['CONST_STANDARD_PATH_ANALYZE'], 'raw_2.data') , 'w') as f:
            f.writelines(str(wynik))
    plotuj(stg, wynik)

In [35]:
stg = {
        'CONST_CLIQUE'    : 3,      #~ Wielkosc kliki
        'CONST_VERTICES'  : 10000,   #~ Ilosc wezlow
        'CONST_OVERRIDEN' : False,  #~ Czy ma nadpisywac pliki podczas zapisywania wynikow
        'CONST_COMPRESS'  : True,   #~ Czy ma kompresowac dane przez zapisem        
        'CONST_DUMP'      : True,   # czy ma zrzucac wektory wynikow 
        'CONST_PATH_BASIC_FOLDER' : 'Wyniki_fazowe',
        'WYN_meanG' : 77
    }

analyze(stg)

([0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56], [0.0, 0.0, 0.0, 0.0, 0.1875, 0.625, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
